### Analyse traffic sensor data to estimate traffic demand for different vehicles

In [39]:
import pandas as pd
import numpy as np
import re

Vorhandene Fahrspuren: S, R

Präfixe: Rechtsabbiegerspuren (R), Geradeausspuren (S)

Fahrzeugklassen: Fahrrad (Bikes), Motorrad (C1), PKW (C2), Kleintransporter (C3), Transporter (C4), LKW (C5)

Source: https://opendata.smartmannheim.de/dataset/traffic-sensor-014

In [40]:
data = pd.read_csv('../src/data/traffic_data/004_014_sensor_trafficData.csv')

In [41]:
data

,timestamps,Bikes,RC1,RC2,RC3,RC4,RC5,SC1,SC2,SC3,SC4,SC5
0,2023-07-26T22:00:00Z,7,0.0,33.0,0.0,10.0,0.0,0,106,0,22,1
1,2023-07-26T23:00:00Z,1,0.0,25.0,0.0,14.0,0.0,0,54,1,5,1
2,2023-07-27T00:00:00Z,1,0.0,13.0,0.0,12.0,0.0,0,37,0,15,1
3,2023-07-27T01:00:00Z,0,0.0,9.0,0.0,11.0,0.0,0,29,2,10,0
4,2023-07-27T02:00:00Z,1,0.0,3.0,0.0,9.0,1.0,0,44,0,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...
137,2023-07-27T17:00:00Z,103,NaN,NaN,NaN,NaN,NaN,6,128,1,77,2
138,2023-07-27T18:00:00Z,70,NaN,NaN,NaN,NaN,NaN,10,78,1,68,4
139,2023-07-27T19:00:00Z,47,NaN,NaN,NaN,NaN,NaN,10,88,4,55,4
140,2023-07-27T20:00:00Z,43,NaN,NaN,NaN,NaN,NaN,13,93,5,46,7


In [42]:
#separate date from timestamp to get demand distribution per hour
timestamp_split = data['timestamps'].apply(lambda x: re.split('T', str(x)))
data['date'] = timestamp_split.apply(lambda x: x[0])
data['time'] = timestamp_split.apply(lambda x: x[1])
data

,timestamps,Bikes,RC1,RC2,RC3,RC4,RC5,SC1,SC2,SC3,SC4,SC5,date,time
0,2023-07-26T22:00:00Z,7,0.0,33.0,0.0,10.0,0.0,0,106,0,22,1,2023-07-26,22:00:00Z
1,2023-07-26T23:00:00Z,1,0.0,25.0,0.0,14.0,0.0,0,54,1,5,1,2023-07-26,23:00:00Z
2,2023-07-27T00:00:00Z,1,0.0,13.0,0.0,12.0,0.0,0,37,0,15,1,2023-07-27,00:00:00Z
3,2023-07-27T01:00:00Z,0,0.0,9.0,0.0,11.0,0.0,0,29,2,10,0,2023-07-27,01:00:00Z
4,2023-07-27T02:00:00Z,1,0.0,3.0,0.0,9.0,1.0,0,44,0,14,0,2023-07-27,02:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2023-07-27T17:00:00Z,103,NaN,NaN,NaN,NaN,NaN,6,128,1,77,2,2023-07-27,17:00:00Z
138,2023-07-27T18:00:00Z,70,NaN,NaN,NaN,NaN,NaN,10,78,1,68,4,2023-07-27,18:00:00Z
139,2023-07-27T19:00:00Z,47,NaN,NaN,NaN,NaN,NaN,10,88,4,55,4,2023-07-27,19:00:00Z
140,2023-07-27T20:00:00Z,43,NaN,NaN,NaN,NaN,NaN,13,93,5,46,7,2023-07-27,20:00:00Z


In [43]:
#replace NaN with 0 to sum up distributions per vehicle type
data = data.fillna(0)

In [44]:
# get combined columns of RC1 and SC1, RC2 and SC2, RC3 and SC3 etc
data['C1'] = data['RC1'] + data['SC1']
data['C2'] = data['RC2'] + data['SC2']
data['C3_4_5'] = data['RC3'] + data['SC3'] + data['RC4'] + data['SC4'] + data['RC5'] + data['SC5'] # small and normal transporter & lkw

In [45]:
# drop previous columns
data.drop(['date','timestamps','RC1', 'SC1', 'RC2', 'SC2', 'RC3', 'SC3', 'RC4', 'SC4', 'RC5', 'SC5'], axis=1, inplace=True)

In [46]:
# move first column to the Last
temp_cols=data.columns.tolist()
new_cols=temp_cols[1:] + temp_cols[0:1]
data=data[new_cols]
data

,time,C1,C2,C3_4_5,Bikes
0,22:00:00Z,0.0,139.0,33.0,7
1,23:00:00Z,0.0,79.0,21.0,1
2,00:00:00Z,0.0,50.0,28.0,1
3,01:00:00Z,0.0,38.0,23.0,0
4,02:00:00Z,0.0,47.0,24.0,1
...,...,...,...,...,...
137,17:00:00Z,6.0,128.0,80.0,103
138,18:00:00Z,10.0,78.0,73.0,70
139,19:00:00Z,10.0,88.0,63.0,47
140,20:00:00Z,13.0,93.0,58.0,43


In [47]:
#group data by time and get average of different vehicle types (Bike RC1 RC2	RC3	RC4	RC5	SC1	SC2	SC3	SC4	SC5)
data.groupby(['time']).mean()

# Fahrrad (Bikes), Motorrad (C1), PKW (C2), Kleintransporter (C3), Transporter (C4), LKW (C5)

,C1,C2,C3_4_5,Bikes
time,,,,
00:00:00Z,0.333333,27.166667,15.166667,5.000000
01:00:00Z,0.333333,19.500000,15.000000,2.333333
02:00:00Z,0.666667,27.333333,16.333333,3.500000
03:00:00Z,1.666667,69.000000,40.833333,16.333333
04:00:00Z,3.166667,163.500000,80.000000,35.833333
05:00:00Z,5.500000,244.500000,107.166667,60.833333
06:00:00Z,4.500000,308.500000,137.833333,110.000000
07:00:00Z,2.333333,302.000000,128.500000,98.333333
08:00:00Z,5.166667,271.166667,123.833333,80.000000


In [48]:
# get sum of every column and divide by 3 to get average of 24 hours get percentage compared to other column C2

data.sum(axis=0) #/data.sum(axis=0)['C2']

time      22:00:00Z23:00:00Z00:00:00Z01:00:00Z02:00:00Z0...
C1                                                    465.0
C2                                                  30857.0
C3_4_5                                              11247.0
Bikes                                                  7567
dtype: object

In [49]:
bikes_percentage = (data['Bikes'].sum()/data['C2'].sum())*100
print('Bike Proportion: ', bikes_percentage)
motorcycle_percentage = (data['C1'].sum()/data['C2'].sum())*100
print('Motorcycle Proportion: ', motorcycle_percentage)
trucks_percentage = (data['C3_4_5'].sum()/data['C2'].sum())*100
print('Truck Proportion: ', trucks_percentage)

Bike Proportion:  24.522798716660727
Motorcycle Proportion:  1.506951421071394
Truck Proportion:  36.44877985546229


Within the simulation:
- Vehicles (68399) will start running at 03:00 am
- Bikes (17100) and Trucks (20500) will start running at 04:00 am
- Motorcycles (1500) will start running at 05:00 am